In [ ]:
SERVICE_ACCOUNT_FILE = 'byui-python-analysis-30a31cf00f2c.json'
REPORT_REQUEST_FILE = 'report-request.json'

In [ ]:
# Libraries
from google.oauth2 import service_account
from googleapiclient.discovery import build
import demjson
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
from pathlib import Path
from glob import glob
import json
import shutil
import base64
import hashlib
import re

In [ ]:
COLUMNS_FILE = './cache/{hash}/columns.csv'
REQUEST_FILE = './cache/{hash}/{date}/info.csv'
SUMMARY_FILE = './cache/{hash}/{date}/summary.csv'
DATA_FILE = './cache/{hash}/{date}/pages/{page}.csv'
CACHE_FILE = './cache/raw/{hash}_{date}_{page}.json'
GA_DATE_FORMAT = '%Y-%m-%d'

def report_to_frame(report):
    # Get metric names as list
    metric_names = [metric.get('name') for metric in report.get('columnHeader').get('metricHeader').get('metricHeaderEntries',[])]    
    metric_names = [re.sub(r'^ga:','', name) for name in metric_names]
    
    # Get dimension names as list
    dimension_names = report.get('columnHeader').get('dimensions',[])
    dimension_names = [re.sub(r'^ga:','', name) for name in dimension_names]
    
    # Collect dimension values into a dataframe
    dimensions = [row.get('dimensions',[]) for row in report.get('data').get('rows',[])]
    dimensions = pd.DataFrame(dimensions, columns = dimension_names)
    
    # Create MultiIndex from dimension values
    index = pd.MultiIndex.from_frame(dimensions) if not dimensions.empty else None
    
    # Collect metric values into list of lists
    rows = []
    num_date_ranges = 1
    for row in report.get('data').get('rows',[]):
        # Concat all metrics for each date range, will seperate later
        values = []
        num_date_ranges = len(row.get('metrics'))
        for date_range in row.get('metrics'):
            values += date_range.get('values')
        rows.append(values)
    
    
    columns = None
    if num_date_ranges > 1:
        # Create column index as (date_ranges x metrics)
        columns = pd.MultiIndex.from_product([range(num_date_ranges), metric_names], names=('date_range','metric'))
    else:
        columns = pd.Index(metric_names)
    
    # Create the DataFrame
    df = pd.DataFrame(rows, index=index, columns=columns)

    # Convert to numbers
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

def report_summary(report):
    metric_names = [re.sub(r'^ga:','', metric.get('name')) for metric in report.get('columnHeader').get('metricHeader').get('metricHeaderEntries')]
    num_date_ranges = len(report.get('data').get('totals'))
    data = {'min':[],'max':[],'sum':[]}
    total_rows = int(report.get('data').get('rowCount',0))
    
    int(report.get('data').get('rowCount',0))
    
    for sums in report.get('data').get('totals'):
        data['sum'] += sums.get('values')
    for sums in report.get('data').get('minimums'):
        data['min'] += sums.get('values')
    for sums in report.get('data').get('maximums'):
        data['max'] += sums.get('values')
        
    index = None
    if num_date_ranges > 1:
        index = pd.MultiIndex.from_product([range(num_date_ranges), metric_names], names=('date_range','metric'))
    else:
        index = pd.Index(metric_names)
    
    df = pd.DataFrame(data, index=index)
    df = df.apply(pd.to_numeric, errors='ignore')
    df['avg'] = df['sum'] / total_rows
#     df = df.transpose()
    
    return df

def report_columns(report):
    dimensions = report.get('columnHeader').get('dimensions',[])
    dimensions = pd.DataFrame(dimensions, columns=['name'])
    dimensions['type'] = 'dimension'
    metrics = report.get('columnHeader').get('metricHeader').get('metricHeaderEntries',[])
    metrics = pd.DataFrame(metrics)
    metrics = metrics.rename(columns={'type':'format'})
    metrics['type'] = 'metric'
    return pd.concat([dimensions, metrics], sort=False)

def report_info(report):
    data = report.get('data')
    nrows = int(data.get('rowCount',0))
    reads = data.get('samplesReadCounts',[nrows])[0]
    space = data.get('samplingSpaceSizes',[nrows])[0]
    return pd.Series({
        'nrows': nrows,
        'reads': reads,
        'space': space,
        'rate': reads/space,
        'golden': data.get('isDataGolden',False),
        'date': data.get('dataLastRefreshed', datetime.now()),
    }).to_frame(name='values')

def hash_dict(d):
    string = json.dumps(d)
    buffer = str.encode(string)
    hashed = hashlib.sha1(buffer).digest()
    encoded = base64.b64encode(hashed).decode('utf-8')
    clean = re.sub(r'\W','',encoded)
    return clean[0:10]

def each_day(date_ranges):
    for date_range in date_ranges:
        start_date = datetime.strptime(date_range.get('startDate'), GA_DATE_FORMAT)
        end_date = datetime.strptime(date_range.get('endDate'), GA_DATE_FORMAT)
        for n in range(int ((end_date - start_date).days)):
            yield (start_date + timedelta(days=n)).strftime(GA_DATE_FORMAT)

def each_request(requests):
    for r in requests:
        request = r.copy()
        
        # Set page size to default to max, so that making less requests
        request['pageSize'] = request.get('pageSize',100000)

        # Set sample size to default to large
        request['samplingLevel'] = request.get('samplingLevel','LARGE')

        # Remove date ranges and page token
        date_ranges = request.pop('dateRanges',[])
        page_token = request.pop('pageToken',0)
        
        hashed = hash_dict(request)

        for date in each_day(date_ranges):
            r = {
                'hash': hashed,
                'date': date,            
                'page': page_token,
                'body': request,
            }
            Path(os.path.dirname(DATA_FILE.format(**r))).mkdir(parents=True, exist_ok=True)
            Path(os.path.dirname(CACHE_FILE.format(**r))).mkdir(parents=True, exist_ok=True)
            yield r
            

def get_report(request):
    file = CACHE_FILE.format(**request)
    
    # Check to see if file is in cache
    if os.path.exists(file):
        with open(file) as f:
            return json.load(f)
    
    # Modify request Body
    body = request['body'].copy()
    body['pageToken'] = str(request['page'])
    body['dateRanges'] = [{
        'startDate':request['date'],
        'endDate':request['date']
    }]
    
    # Make the request
    print('{hash}\t{date}\t{page}'.format(**request))
    r = analytics.reports().batchGet(body={'reportRequests':[body]}).execute()
    report = r.get('reports')[0]
    
    with open(file,'w') as f:
        json.dump(report, f)
    
    return report

def write_csvs(request, report):
    if not os.path.exists(COLUMNS_FILE.format(**request)):
        columns = report_columns(report)
        columns.index.name = 'row'
        columns = columns.reset_index()
        columns['request'] = request['hash']
        columns = columns.set_index(['request','row'])
        columns.to_csv(COLUMNS_FILE.format(**request))
    
    if not os.path.exists(REQUEST_FILE.format(**request)):
        info = report_info(report)
        info['request'] = request['hash']
        info['date'] = request['date']
        info = info.set_index(['request','date'])
        info.to_csv(REQUEST_FILE.format(**request))
    
    if not os.path.exists(SUMMARY_FILE.format(**request)):
        summary = report_summary(report)
        summary.index.name = 'metric'
        summary = summary.reset_index()
        summary['request'] = request['hash']
        summary['date'] = request['date']
        summary = summary.set_index(['request','date','metric'])
        summary.to_csv(SUMMARY_FILE.format(**request))

    if not os.path.exists(DATA_FILE.format(**request)):
        report_to_frame(report).to_csv(DATA_FILE.format(**request), header=False)

def concat_files(sources, destination):
    with open(destination,'ab') as wfd:
        for f in sources:
            with open(f,'rb') as fd:
                shutil.copyfileobj(fd, wfd)

### Step 1: Initalize API with Service Account File

In [ ]:
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/analytics.readonly'])
analytics = build('analyticsreporting', 'v4', credentials=credentials)

### Step 2: Request All Reports

In [ ]:
rids = set()
with open(REPORT_REQUEST_FILE) as f:
    requests = demjson.decode(f.read())
    
    for request in each_request(requests):
        
        rids.add(request['hash'])
        report = get_report(request)
        write_csvs(request, report)
        
        while 'nextPageToken' in report:
            request['page'] = report['nextPageToken']
            report = get_report(request)
            write_csvs(request, report)

### Step 3: Concat Reports

In [ ]:
run_time = datetime.now().strftime("%h%d_%H%M")
for rid in rids:
    file = './{}_{}.csv'.format(run_time, rid)
    columns = pd.read_csv('./cache/{}/columns.csv'.format(rid))
    pd.DataFrame(columns=columns['name']).to_csv(file, index=False)
    concat_files(glob('./cache/{}/*/pages/*.csv'.format(rid)), file)